In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from surprise import Reader, Dataset, evaluate, print_perf, GridSearch
from surprise import SVD, BaselineOnly, Prediction, accuracy
from sklearn.metrics import roc_auc_score

random.seed(561)

In [2]:
# users = pd.read_csv('~/Columbia/Personalization Theory/lastfm-dataset-1K/userid-profile.tsv', header=None)
data = pd.read_csv('~/Columbia/Personalization Theory/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv',
                   delimiter="\t", header=None,
                   names = ["userid","timestamp","artistid",
                            "artistname","trackid","trackname"])

In [3]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

For the mini-project, we are using a smaller dataset. The following transformations will convert the dataset to use number of plays as our metric, grouped by user and artist.

To help with our data cleaning and setting up the matrices, we used [this website](https://jessesw.com/Rec-System/) to guide us.

In [4]:
data = data.groupby(['userid', 'artistname']).size().reset_index(name='plays')

In [5]:
users = list(np.sort(data.userid.unique())) # Get our unique users
artists = list(data.artistname.unique()) # Get our unique artists
quantity = list(data.plays) # All of our plays

rows = data.userid.astype('category', categories = users).cat.codes 
# Get the associated row indices
cols = data.artistname.astype('category', categories = artists).cat.codes 
# Get the associated column indices
plays_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(users), len(quantity)))
plays_sparse

<992x897421 sparse matrix of type '<class 'numpy.int64'>'
	with 897421 stored elements in Compressed Sparse Row format>

In [6]:
# Sparsity of the matrix
matrix_size = plays_sparse.shape[0]*plays_sparse.shape[1] # Number of possible interactions in the matrix
num_plays = len(plays_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_plays/matrix_size))
sparsity

99.8991935483871

## Dataset Reduction

The data set has a sparsity of 99.899%, which is very low even for matrices that are intended to be sparse. We experimented with removing rare artists and including only the top n artists, which had minimal effect on the sparsity.

In [7]:
#rare_artists = data.query("plays < 6"). \
#    groupby('artistname').size().reset_index(name='users_listening_to_artist'). \
#    query("users_listening_to_artist < 10")
    
top5000_artists = data.groupby('artistname')['plays'].sum().reset_index(name='plays'). \
    nlargest(5000,'plays')

In [9]:
reduced_data = data[data.artistname.isin(top5000_artists['artistname'])]

print(reduced_data.shape, data.shape)

(424743, 3) (897421, 3)


In [10]:
users = list(np.sort(reduced_data.userid.unique())) # Get our unique users
artists = list(reduced_data.artistname.unique()) # Get our unique artists
quantity = list(reduced_data.plays) # All of our plays

rows = reduced_data.userid.astype('category', categories = users).cat.codes 
# Get the associated row indices
cols = reduced_data.artistname.astype('category', categories = artists).cat.codes 
# Get the associated column indices
plays_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(users), len(quantity)))

plays_sparse

<991x424743 sparse matrix of type '<class 'numpy.int64'>'
	with 424743 stored elements in Compressed Sparse Row format>

In [11]:
# Sparsity of the matrix
matrix_size = plays_sparse.shape[0]*plays_sparse.shape[1] # Number of possible interactions in the matrix
num_plays = len(plays_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_plays/matrix_size))
sparsity

99.89909182643795

In [12]:
# free up memory
del users, artists, quantity, rows, cols, plays_sparse, matrix_size, num_plays, sparsity

## Data Preprocessing

For the SVD algorithm, we use the Surprise package. We modify the plays parameter to be a binary indicator of whether someone has listened to an artist.

Including every artist would result in a dataset with over 173 million rows. We chose to limit the data to the top 5000 artists by number of plays, which brings the dataset down to 5 million rows.

In [13]:
# Normalize plays to be a percentage of total plays by artist
# usertotal = data.groupby('userid')['plays'].sum().reset_index(name="total_plays")
# normalized_data = pd.merge(reduced_data, usertotal)
# normalized_data['normalized_plays'] = normalized_data['plays']/normalized_data['total_plays']
# normalized_data.drop(['total_plays'], inplace=True, axis=1)
# normalized_data.loc[normalized_data['plays'] != 0, 'plays'] = 1

# set to binary of whether a user listed to an artist
data.loc[data['plays'] != 0, 'plays'] = 1
# remove all artists not in the top 5000
data = data[data.artistname.isin(top5000_artists['artistname'])]

# Add all user-artist combos, with no plays = 0
data = data.pivot(index='userid', columns='artistname', values='plays').fillna(0).reset_index()
data = data.melt(id_vars=['userid'], var_name=['artistname'])
data = data.rename(columns = {'value':'plays'})

print(len(data))
data.head()

4955000


,userid,artistname,plays
0,user_000001,!!!,0.0
1,user_000002,!!!,0.0
2,user_000003,!!!,0.0
3,user_000004,!!!,1.0
4,user_000005,!!!,0.0


## Baseline

We will use a simple algorithm to determine a benchmark for our SVD model. From the `Surprise` package, the baseline model is a simple alternating least squares.

In [16]:
reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
model_data = Dataset.load_from_df(data, reader)
model_data.split(n_folds=3)

# We'll use the famous SVD algorithm.
algo = BaselineOnly(bsl_options = {'method': 'als'})

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, model_data, measures=['RMSE', 'MAE'])

# predictions = predict(data['userid'], data['artistname'], data['plays'])
print_perf(perf)

Evaluating RMSE, MAE of algorithm BaselineOnly.

------------
Fold 1
Estimating biases using als...
RMSE: 0.2546
MAE:  0.1365
------------
Fold 2
Estimating biases using als...
RMSE: 0.2547
MAE:  0.1367
------------
Fold 3
Estimating biases using als...
RMSE: 0.2546
MAE:  0.1366
------------
------------
Mean RMSE: 0.2547
Mean MAE : 0.1366
------------
------------
        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.2546  0.2547  0.2546  0.2547  
MAE     0.1365  0.1367  0.1366  0.1366  


## SVD

We used GridSearch to tune our hyperparameters for our SVD model. We investigated including the 'epoch' parameter, but it drastically increased the run time of the code below. Instead, we chose to keep the default number of epochs and tune the other 3 parameters.

In [15]:
reader = Reader(rating_scale=(0, 1))

param_grid = {'n_factors': np.arange(60,140,20),
              'lr_all': np.arange(0.002,0.014, 0.004),'reg_all': np.arange(0.02,0.1, 0.02)}
grid_search = GridSearch(SVD, param_grid, measures=['RMSE'])

model_data = Dataset.load_from_df(data[['userid', 'artistname', 'plays']], reader)
model_data.split(n_folds=3)

grid_search.evaluate(model_data)

results_df = pd.DataFrame.from_dict(grid_search.cv_results)

[{'n_factors': 60, 'lr_all': 0.002, 'reg_all': 0.02}, {'n_factors': 60, 'lr_all': 0.002, 'reg_all': 0.040000000000000001}, {'n_factors': 60, 'lr_all': 0.002, 'reg_all': 0.059999999999999998}, {'n_factors': 60, 'lr_all': 0.002, 'reg_all': 0.080000000000000002}, {'n_factors': 60, 'lr_all': 0.0060000000000000001, 'reg_all': 0.02}, {'n_factors': 60, 'lr_all': 0.0060000000000000001, 'reg_all': 0.040000000000000001}, {'n_factors': 60, 'lr_all': 0.0060000000000000001, 'reg_all': 0.059999999999999998}, {'n_factors': 60, 'lr_all': 0.0060000000000000001, 'reg_all': 0.080000000000000002}, {'n_factors': 60, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 60, 'lr_all': 0.01, 'reg_all': 0.040000000000000001}, {'n_factors': 60, 'lr_all': 0.01, 'reg_all': 0.059999999999999998}, {'n_factors': 60, 'lr_all': 0.01, 'reg_all': 0.080000000000000002}, {'n_factors': 80, 'lr_all': 0.002, 'reg_all': 0.02}, {'n_factors': 80, 'lr_all': 0.002, 'reg_all': 0.040000000000000001}, {'n_factors': 80, 'lr_all': 0.002, 'r

params:  {'n_factors': 100, 'lr_all': 0.0060000000000000001, 'reg_all': 0.059999999999999998}
------------
Mean RMSE: 0.2550
------------
------------
Parameters combination 32 of 48
params:  {'n_factors': 100, 'lr_all': 0.0060000000000000001, 'reg_all': 0.080000000000000002}
------------
Mean RMSE: 0.2554
------------
------------
Parameters combination 33 of 48
params:  {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.02}
------------
Mean RMSE: 0.2357
------------
------------
Parameters combination 34 of 48
params:  {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.040000000000000001}
------------
Mean RMSE: 0.2488
------------
------------
Parameters combination 35 of 48
params:  {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.059999999999999998}
------------
Mean RMSE: 0.2548
------------
------------
Parameters combination 36 of 48
params:  {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.080000000000000002}
------------
Mean RMSE: 0.2558
------------
------------
Parameters combina

## Results

The optimal parameters to reduce RMSE are as follows:
- `factors` = 120,
- `learning rate` = 0.01,
- `regularization` = 0.02.

In [17]:
#reader = Reader(rating_scale=(0, 1))

# The columns must correspond to user id, item id and ratings (in that order).
#model_data = Dataset.load_from_df(data, reader)
#model_data.split(n_folds=3)

# We'll use the famous SVD algorithm.
algo = SVD(n_factors = 120, lr_all = 0.01, reg_all = 0.02)

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, model_data, measures=['RMSE', 'MAE'])

# predictions = predict(data['userid'], data['artistname'], data['plays'])
print_perf(perf)

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.2355
MAE:  0.1197
------------
Fold 2
RMSE: 0.2355
MAE:  0.1200
------------
Fold 3
RMSE: 0.2354
MAE:  0.1201
------------
------------
Mean RMSE: 0.2355
Mean MAE : 0.1199
------------
------------
        Fold 1  Fold 2  Fold 3  Mean    
RMSE    0.2355  0.2355  0.2354  0.2355  
MAE     0.1197  0.1200  0.1201  0.1199  


## Analysis

To investigate how tuning impacts our loss functions, we iteratively varied one hyperparameter to see how RMSE changes while others are held constant.

For example, we held the learning rate and regularization at their respective optimal values of 0.01 and 0.08, while varying the number of factors from 20 to 120 in increments of 20.

We saved the results into three files: reg.csv, learning.csv and factor.csv.

In [25]:
param_grid = {'n_factors': np.arange(60,140,8),'lr_all': [.01],'reg_all': [.02]}
grid_search = GridSearch(SVD, param_grid, measures=['RMSE'])

#model_data = Dataset.load_from_df(data[['userid', 'artistname', 'plays']], reader)
#model_data.split(n_folds=3)

grid_search.evaluate(model_data)

results_df = pd.DataFrame.from_dict(grid_search.cv_results)

[{'n_factors': 60, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 68, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 76, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 84, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 92, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 100, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 108, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 116, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 124, 'lr_all': 0.01, 'reg_all': 0.02}, {'n_factors': 132, 'lr_all': 0.01, 'reg_all': 0.02}]
------------
Parameters combination 1 of 10
params:  {'n_factors': 60, 'lr_all': 0.01, 'reg_all': 0.02}
------------
Mean RMSE: 0.2362
------------
------------
Parameters combination 2 of 10
params:  {'n_factors': 68, 'lr_all': 0.01, 'reg_all': 0.02}
------------
Mean RMSE: 0.2359
------------
------------
Parameters combination 3 of 10
params:  {'n_factors': 76, 'lr_all': 0.01, 'reg_all': 0.02}
------------
Mean RMSE: 0.2358
------------
------------
Parameters combin

In [26]:
results_df.to_csv('../data/factors.csv', sep='\t')

## AUC

We also use AUC to evaluate our model.

In [ ]:
for trainset, testset in model_data.folds():

    # train and test algorithm.
    algo.train(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error and Mean Absolute Error
    #rmse = accuracy.rmse(predictions, verbose=True)
    #mae = accuracy.mae(predictions, verbose=True)

In [ ]:
output = pd.DataFrame(predictions)
output = output.drop(['r_ui', 'details'], axis=1)

combined = pd.merge(data, output, how='right', left_on=['userid','artistname'], right_on=['uid','iid'])
combined = combined.drop(['uid', 'iid'], axis=1).set_index('userid')
combined['predicted'] = np.where(combined['est']>0.5, 1, 0)
combined['plays'] = np.where(np.isnan(combined['plays']),0,combined['plays'])
combined.describe()

#fpr, tpr, thresholds = metrics.roc_curve(combined['normalized_plays'], combined['est'], pos_label=2)
#metrics.auc(fpr, tpr)

#roc_auc_score(combined['plays'],combined['est'])

In [ ]:
sorted(predictions)